# 4.10 Tasks part 1

#### 1. import libraries and dataframes

#### 2. Address PII data

#### 3. Regional segmentation

#### 4. Exclusion of low activity customers

#### 5. Customer profiling

#### 6. Compare customer profiles with regions

#### 7. Compare customer profiles with departments

## 1. Import libraries and dataframes

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [ ]:
#set scripts path in variable 'path'
path = r'/Users/constantinmelachrinos/Documents/careerfoundry data analyst/Course files/Data Analytics Immersion/Achievement 4 - InstaCart - Python/Instacart Basket Analysis'

#import Instacart data_sets
ords_prods_cust = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_customers_combined.pkl'))
df_departments = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'departments_wrangled.csv'))

# 2. Address PII data

In [ ]:
#first have a look at the dataframe for completness
ords_prods_cust.head()

### PII data which has to be addressed are first and last names in the dataframe. These columns have to be deleted.

In [ ]:
# dropping columns 'first_name', 'last_name' and '_merge' columns
ords_prods_cust=ords_prods_cust.drop(['first_name','last_name','_merge'], axis=1)

In [ ]:
#check if executed correctly
ords_prods_cust.columns

# 3. Regional segmentation 
### 3.1 Create a regional segmentation of the data. You’ll need to create a “Region” column based on the “State” column from your customers data set.

In [ ]:
#Create list of regions.First Northeast
northeast=['Maine','New Hampshire','Vermont','Massachusetts','Rhode Island','Connecticut','New York','Pennsylvania','New Jersey']

In [ ]:
#Create list of regions. Second Midwest
midwest=['Wisconsin','Michigan','Illinois','Indiana','Ohio','North Dakota','South Dakota','Nebraska','Kansas','Minnesota','Iowa','Missouri']

In [ ]:
#Create list of regions. Third South
south=['Delaware','Maryland','District of Columbia','Virginia','West Virginia','North Carolina','South Carolina','Georgia','Florida','Kentucky','Tennessee','Mississippi','Alabama','Oklahoma','Texas','Arkansas','Louisiana']

In [ ]:
#Create list of regions. Fourth West
west=['Idaho','Montana','Wyoming','Nevada','Utah','Colorado','Arizona','New Mexico','Alaska','Washington','Oregon','California','Hawaii']

In [ ]:
#loc function for regional segmentation by producing an additional column named 'region'
ords_prods_cust.loc[ords_prods_cust['state'].isin(northeast), 'region'] = 'Northeast'
ords_prods_cust.loc[ords_prods_cust['state'].isin(midwest), 'region'] = 'Midwest'
ords_prods_cust.loc[ords_prods_cust['state'].isin(south), 'region'] = 'South'
ords_prods_cust.loc[ords_prods_cust['state'].isin(west), 'region'] = 'West'

In [ ]:
#Check frquency of the different regions
ords_prods_cust['region'].value_counts(dropna=False)

### 3.2 Determine whether there’s a difference in spending habits between the different U.S. regions.

In [ ]:
#crosstab the 'region' column with the 'spending_behaviour' column
crosstab = pd.crosstab(ords_prods_cust['region'], ords_prods_cust['spending_behaviour'], dropna = False)

In [ ]:
#check outcome
crosstab

# 4. Exclusion of low activity customers
### 4.1 The Instacart CFO isn’t interested in customers who don’t generate much revenue for the app. Create an exclusion flag for low-activity customers (customers with less than 5 orders) and exclude them from the data.

In [ ]:
#check max order column
ords_prods_cust[['max_order']]

In [ ]:
# Create flag on basis of 'max_order' column.
ords_prods_cust.loc[ords_prods_cust['max_order'] < 5, 'activity'] = 'low'
ords_prods_cust.loc[ords_prods_cust['max_order'] >= 5, 'activity'] = 'high'

In [ ]:
#exclude low activity customers in extra data frame called df_low_activity
df_low_activity = ords_prods_cust[ords_prods_cust['activity']=='low']

In [ ]:
df_low_activity.head()

### 4.2 Export low activity sample data frame

In [ ]:
#export dataframe
df_low_activity.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'customers_low_activity.pkl'))

# 5. Customer profiling
### 5.1.1 Age profiling (Young, mid-aged, senior)
### 5.1.2 Create an appropriate visualization to show the distribution of profiles.
### 5.1.3 Aggregate the max, mean, and min variables on a customer-profile level for usage frequency and expenditure.

### 5.1.1 Age profiling (Young, mid-aged, senior)

In [ ]:
# Create flag on basis of 'age' column.
ords_prods_cust.loc[ords_prods_cust['age'] <= 30 , 'age_group'] = 'Young'
ords_prods_cust.loc[(ords_prods_cust['age'] > 30) & (ords_prods_cust['age'] < 60),  'age_group'] = 'Mid-age'
ords_prods_cust.loc[ords_prods_cust['age'] >= 60 , 'age_group'] = 'Senior'

In [ ]:
# frequency of each age group
ords_prods_cust['age_group'].value_counts(dropna=False)

### 5.1.2 Create an appropriate visualization to show the distribution of profiles.

In [ ]:
#create bar chart distribution of age groups

#define chart size
plt.figure(figsize=(10, 6))

# define Seaborn color palette to use 
palette_color = ['#228B22','#32CD32','#90EE90']

#define column chart
bar_comp_age=sns.countplot(x = 'age_group', 
            data = ords_prods_cust, 
            palette=palette_color)
bar_comp_age.set(
            xlabel='Age groups',
            ylabel='Frequency in Millions')

#format the ticks on the y-axis since the frequency is in Millions
ylabels = ['{:,.2f}'.format(y) + 'M' for y in bar_comp_age.get_yticks()/1000000]
bar_comp_age.set_yticklabels(ylabels)

plt.title('Distribution of Age Groups', weight='bold').set_fontsize('18')
#export visualization
plt.savefig(os.path.join(path, '04 Analysis','Visualizations', 'bar_age_goups.png'))

### 5.1.3 Aggregate the max, mean, and min variables on a customer-profile level for usage frequency and expenditure.

In [ ]:
#Aggregate the usage frequency
ords_prods_cust.groupby('age_group').agg({'median_days_since_prior_order':['max', 'mean', 'min']})

In [ ]:
#Aggregate the spending
ords_prods_cust.groupby('age_group').agg({'prices':['max', 'mean', 'min']})

In [ ]:
ords_prods_cust.groupby('age_group').agg({'order_hour_of_day':['max','mean','min']})

### 5.2.1 Income profiling

In [ ]:
# Create flag on basis of 'income' column.
ords_prods_cust.loc[ords_prods_cust['income'] <= 48000 , 'level_income'] = 'Low income'
ords_prods_cust.loc[(ords_prods_cust['income'] > 48000) & (ords_prods_cust['income'] < 145000),  'level_income'] = 'Middle income'
ords_prods_cust.loc[ords_prods_cust['income'] >= 145000 , 'level_income'] = 'High income'

### source of differentiating the income level see here:
### https://www.pewresearch.org/short-reads/2020/07/23/are-you-in-the-american-middle-class/#:~:text=In%202018%2C%20the%20national%20middle,(incomes%20in%202018%20dollars)

In [ ]:
# frequency of each income level group
ords_prods_cust['level_income'].value_counts(dropna=False)

### 5.2.2 Create an appropriate visualization to show the distribution of profiles.

In [ ]:
# declaring exploding pie 
explode = [0.1, 0, 0] 
# define Seaborn color palette to use 
palette_color = ['#228B22','#32CD32','#90EE90']
# plotting data on chart
plt.figure(figsize=(10, 6))
plt.pie(ords_prods_cust['level_income'].value_counts(), 
        labels=ords_prods_cust['level_income'].value_counts().index, 
        colors=palette_color, 
        explode=explode, 
        autopct='%.0f%%',
        wedgeprops = {'edgecolor' : 'black'})
plt.title('Distribution of Customers by Income',weight='bold').set_fontsize('18')
#export visualization
plt.savefig(os.path.join(path, '04 Analysis','Visualizations', 'pie_income.png'))

### 5.2.3 Aggregate the max, mean, and min variables on a customer-profile level for usage frequency and expenditure.

In [ ]:
#Aggregate the usage frequency
ords_prods_cust.groupby('level_income').agg({'median_days_since_prior_order':['max', 'mean', 'min']})

In [ ]:
#Aggregate the spending
ords_prods_cust.groupby('level_income').agg({'prices':['max', 'mean', 'min']})

### 5.3.1 Certain goods in 'department_id' column profiling

In [ ]:
#check departments
df_departments

#### Since i am an animal lover i would like to profile customers on their buying behaviour how many of the customers have a pet at home

In [ ]:
# Create flag on basis of 'department_id' = 7 column.
ords_prods_cust.loc[ords_prods_cust['department_id'] == 8 , 'pets'] = 'Pets'
ords_prods_cust.loc[ords_prods_cust['department_id'] != 8 , 'pets'] = 'No pets'

In [ ]:
# frequency of each 'pets' group
ords_prods_cust['pets'].value_counts(dropna=False)

### 5.3.2 Create an appropriate visualization to show the distribution of profiles.

In [ ]:
# declaring exploding pie 
explode = [0.1, 0]
# define Seaborn color palette to use 
palette_color = ['#228B22','#32CD32']
# plotting data on chart
plt.figure(figsize=(10, 6))
plt.pie(ords_prods_cust['pets'].value_counts(), 
        labels=ords_prods_cust['pets'].value_counts().index, 
        colors=palette_color, 
        explode=explode, 
        autopct='%.0f%%',
        wedgeprops = {'edgecolor' : 'black'})
plt.title('Instacart Distribution of Customers buying pet products',weight='bold').set_fontsize('18')
#export visualization
plt.savefig(os.path.join(path, '04 Analysis','Visualizations', 'pie_pets.png'))

### 5.3.3 Aggregate the max, mean, and min variables on a customer-profile level for usage frequency and expenditure.

In [ ]:
#Aggregate the usage frequency
ords_prods_cust.groupby('pets').agg({'median_days_since_prior_order':['max', 'mean', 'min']})

In [ ]:
#Aggregate the spending
ords_prods_cust.groupby('pets').agg({'prices':['max', 'mean', 'min']})

### 5.4.1 Profiling customers on dependants

In [ ]:
# Create flag on basis of 'dependants' column.
ords_prods_cust.loc[ords_prods_cust['n_dependants'] > 0 , 'dependants'] = 'Has dependants'
ords_prods_cust.loc[ords_prods_cust['n_dependants'] == 0 , 'dependants'] = 'No dependants'

In [ ]:
# frequency of each 'dependants' group
ords_prods_cust['dependants'].value_counts(dropna=False)

### 5.4.2 Create an appropriate visualization to show the distribution of profiles.

In [ ]:
# declaring exploding pie 
explode = [0.1, 0]
# define Seaborn color palette to use 
palette_color = ['#228B22','#32CD32']
# plotting data on chart
plt.figure(figsize=(10, 6))
plt.pie(ords_prods_cust['dependants'].value_counts(), 
        labels=ords_prods_cust['dependants'].value_counts().index, 
        colors=palette_color, 
        explode=explode, 
        autopct='%.0f%%',
        wedgeprops = {'edgecolor' : 'black'})
plt.title('Instacart Distribution of Customers based on Dependants',weight='bold').set_fontsize('18')
#export visualization
plt.savefig(os.path.join(path, '04 Analysis','Visualizations', 'pie_dependants.png'))

### 5.4.3 Aggregate the max, mean, and min variables on a customer-profile level for usage frequency and expenditure.

In [ ]:
#Aggregate the usage frequency
ords_prods_cust.groupby('dependants').agg({'median_days_since_prior_order':['max', 'mean', 'min']})

In [ ]:
#Aggregate the spending
ords_prods_cust.groupby('dependants').agg({'prices':['max', 'mean', 'min']})

# 6. Compare customer profiles with regions and departments


In [ ]:
#copy only relevant columns in sample dataframe to save time and processor capacity
df_sample = ords_prods_cust[['user_id', 'age_group', 'level_income', 'dependants', 'region', 'pets']]

## 6.1 Compare customer profiles with regions

### 6.1.1 Age groups

In [ ]:
#create bar chart with comparison of age groups in different regions

plt.figure(figsize=(10, 6))
# define Seaborn color palette to use 
palette_color = ['#228B22','#32CD32','#90EE90']
#define column chart
bar_comp_age=sns.countplot(x = 'region', 
            hue = 'age_group',
            data = df_sample, 
            palette=palette_color)
bar_comp_age.set(
            xlabel='Regions',
            ylabel='Frequency')
plt.title('Comparison Age Groups vs Regions', weight='bold').set_fontsize('18')
#export visualization
plt.savefig(os.path.join(path, '04 Analysis','Visualizations', 'bar_comp_age_regions.png'))

### Conclusion: 
#### over all regions most customers are in the mid aged group. Follow up are the seniors. And on third place young people. Why young people on third place may could mean that they dont earn as much as the mid-aged and seniors and/or still live at home and dont have to buy the groceries themselves.

### 6.1.2 Income groups

In [ ]:
#create bar chart with comparison of age groups in different regions

plt.figure(figsize=(10, 6))
# define Seaborn color palette to use 
palette_color = ['#228B22','#32CD32','#90EE90']
#define column chart
bar_comp_age=sns.countplot(x = 'region', 
            hue = 'level_income',
            data = df_sample, 
            palette=palette_color)
bar_comp_age.set(
            xlabel='Regions',
            ylabel='Frequency')
plt.title('Comparison Income groups in different Regions', weight='bold').set_fontsize('18')
#export visualization
plt.savefig(os.path.join(path, '04 Analysis','Visualizations', 'bar_comp_income_regions.png'))

### Conclusion: 
#### over all regions most customers are in the middle income group which makes the majority of the customers. This makes total sence and displays a healthy economy where the middle class rules the majority of the market, which is an indication of a strong economy in the country.

### 6.1.3 Pet lover groups

In [ ]:
#create bar chart with comparison of pet lovers in different regions

plt.figure(figsize=(10, 6))
# define Seaborn color palette to use 
palette_color = ['#228B22','#32CD32','#90EE90']
#define column chart
bar_comp_age=sns.countplot(x = 'region', 
            hue = 'pets',
            data = df_sample, 
            palette=palette_color)
bar_comp_age.set(
            xlabel='Regions',
            ylabel='Frequency')
plt.title('Comparison Pet lovers in different Regions', weight='bold').set_fontsize('18')
#export visualization
plt.savefig(os.path.join(path, '04 Analysis','Visualizations', 'bar_comp_pet_regions.png'))

### Conclusion: 
#### All over the regions the majority of people dont have pets or dont buy pet products.

### 6.1.4 Dependants groups

In [ ]:
#create bar chart with comparison of dependants in different regions

plt.figure(figsize=(10, 6))
# define Seaborn color palette to use 
palette_color = ['#228B22','#32CD32','#90EE90']
#define column chart
bar_comp_age=sns.countplot(x = 'region', 
            hue = 'dependants',
            data = df_sample, 
            palette=palette_color)
bar_comp_age.set(
            xlabel='Regions',
            ylabel='Frequency')
plt.title('Comparison Dependants in different Regions', weight='bold').set_fontsize('18')
#export visualization
plt.savefig(os.path.join(path, '04 Analysis','Visualizations', 'bar_comp_dependants_regions.png'))

### Conclusion:
#### All over all regions the most customers have dependants. The most are in the South and the least in the Northeast. 

### Overall conclusion of the findings in the above visualizations:
#### The overall picture is quite proportional to eachother and there is no significant trend or insight. Maybe if we compare the population count in each region to the customers in percentage and rerun the analysis it could be that we could find some trend.

## 7. Compare customer profiles with departments

## 7.1 Merge dataframe with departments dataframe
### Before we can go any further with producing visuals we need to merge the department dataframe with the ords_prods_cust dataframe to analyse the comparison between customers with the different departments 

In [ ]:
#check departments
df_departments.head()

In [ ]:
#check types in department_id dataframe
df_departments.dtypes

In [ ]:
#rename column unnamed to department_id
df_departments.rename(columns={'Unnamed: 0': 'department_id'},inplace=True)

In [ ]:
#check if types of departmen_id in both dataframes are the same
df_departments.head()

In [ ]:
ords_prods_cust['department_id'].dtypes == df_departments['department_id'].dtypes

In [ ]:
ords_prods_cust['department_id'].value_counts(dropna=False)

In [ ]:
#Merge the two dataframes on department_id
complete_data = ords_prods_cust.merge(df_departments, on = 'department_id', indicator = True, how = 'inner')

In [ ]:
#Check if merge was succesful
complete_data['department'].value_counts(dropna=False)

In [ ]:
# Drop '_merge' flag
complete_data=complete_data.drop('_merge', axis = 1)

In [ ]:
complete_data.head()

## 7.2 Compare age groups with departments

In [ ]:
#create bar chart with comparison of age groups to departments. Show the top 10 departments per age group

plt.figure(figsize=(10, 6))
# define Seaborn color palette to use 
palette_color = ['#228B22','#32CD32','#90EE90']
#define column chart
bar_age_department=sns.countplot(y = 'department', 
            hue = 'age_group',
            data = complete_data, 
            palette=palette_color,
            order = complete_data['department'].value_counts().iloc[:10].index)
bar_age_department.set(
            xlabel='Counts',
            ylabel='Departments')
plt.title('Comparison Age Groups and Departments', weight='bold').set_fontsize('18')
#export visualization
plt.savefig(os.path.join(path, '04 Analysis','Visualizations', 'bar_compe_age_departments.png'))

### Conclusion: No real difference between the age groups compared to the departments. I would have expected that there would be a significant difference between the age groups. Young people tend to eat more unhealthy and cheap food.

## 7.3 Compare income with departments

In [ ]:
sample_data_low_income=complete_data[complete_data['level_income']=='Low income']

In [ ]:
#create bar chart showing comparison of low income customers on departments

plt.figure(figsize=(10, 6))
# define Seaborn color palette to use 
palette_color = ['#32CD32']
#define column chart
bar_department=sns.countplot(y = 'department',
            data = sample_data_low_income, 
            palette=palette_color,
            order = sample_data_low_income['department'].value_counts().iloc[:10].index)
bar_department.set(
            xlabel='Counts',
            ylabel='Departments')
plt.title('Comparison low income customers with Departments', weight='bold').set_fontsize('18')
#export visualization
plt.savefig(os.path.join(path, '04 Analysis','Visualizations', 'bar_comp_low_income_departments.png'))

In [ ]:
#Save sample data of middle income customers in sample dataframe
sample_data_middle_income=complete_data[complete_data['level_income']=='Middle income']

In [ ]:
#create bar chart showing comparison of middle income customers on departments

plt.figure(figsize=(10, 6))
# define Seaborn color palette to use 
palette_color = ['#32CD32']
#define column chart
bar_department=sns.countplot(y = 'department',
            data = sample_data_middle_income, 
            palette=palette_color,
            order = sample_data_middle_income['department'].value_counts().iloc[:10].index)
bar_department.set(
            xlabel='Counts',
            ylabel='Departments')
plt.title('Comparison middle income customers with Departments', weight='bold').set_fontsize('18')
#export visualization
plt.savefig(os.path.join(path, '04 Analysis','Visualizations', 'bar_comp_mid_income_departments.png'))

In [ ]:
#Save sample data of middle income customers in sample dataframe
sample_data_high_income=complete_data[complete_data['level_income']=='High income']

In [ ]:
#create bar chart showing comparison of middle income customers on departments

plt.figure(figsize=(10, 6))
# define Seaborn color palette to use 
palette_color = ['#32CD32']
#define column chart
bar_department=sns.countplot(y = 'department',
            data = sample_data_high_income, 
            palette=palette_color,
            order = sample_data_high_income['department'].value_counts().iloc[:10].index)
bar_department.set(
            xlabel='Counts',
            ylabel='Departments')
plt.title('Comparison high income customers with Departments', weight='bold').set_fontsize('18')
#export visualization
plt.savefig(os.path.join(path, '04 Analysis','Visualizations', 'bar_comp_high_income_departments.png'))

### Conclusion:
#### As the three visualizations show do they have the similar top 5 departments. On all three income groups produce is on first place. But second place with low income groups is snacks. Where with higher income groups its dairy and beverages products. Which shows the lower income groups go for cheaper and unhealthy products.

## 7.4 Compare income with departments

In [ ]:
#create bar chart with comparison of dependants to departments. Show the top 10 departments per dependants group

plt.figure(figsize=(10, 6))
# define Seaborn color palette to use
palette_color = ['#228B22','#32CD32']
#define column chart
bar_dependants_department=sns.countplot(y = 'department', 
            hue = 'dependants',
            data = complete_data, 
            palette=palette_color,
            order = complete_data['department'].value_counts().iloc[:10].index)
bar_dependants_department.set(
            xlabel='Counts',
            ylabel='Departments')
plt.title('Comparison Dependants Groups with Departments', weight='bold').set_fontsize('18')
#export visualization
plt.savefig(os.path.join(path, '04 Analysis','Visualizations', 'bar_comp_dependants_departments.png'))

#### no real difference in products between people with dependants or without. Just the amount is more but we know already that most people have at least one dependant.

## 8. Export final dataframe set

In [ ]:
#export complete dataframe set to drive
complete_data.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'instacart_final_data.pkl'))